In [1]:
#!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import time
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from pins import board_rsconnect
from collections import Counter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 


DistributionNotFound: The 'google-cloud-bigquery-storage' distribution was not found and is required by the application

In [2]:
def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()

# given skill/combination of skills in iterable object 
# these function return total numbers of developers are cluster distribution in a series format

def avs_supply_passed_mcq(skill):
    temp_data = dev_data                            
    for s in skill:
        temp_data = temp_data.loc[temp_data['passed_skill_id_set'].apply(lambda x:s in x)]
    return temp_data['cluster_label'].value_counts().append(pd.Series(temp_data.shape[0], index = ['total_devs']))

def avs_supply_self_skill(skill):
    temp_data = dev_data                            
    for s in skill:
        temp_data = temp_data.loc[temp_data['self_skill_id_set'].apply(lambda x:s in x)]
    return temp_data['cluster_label'].value_counts().append(pd.Series(temp_data.shape[0], index = ['total_devs']))

#print('\n Showing total number of developers for a given skill using above function')
#avs_supply_passed_mcq([120])

## Loading Data from GBQ/r-pins

In [3]:
job_mcq_must = """with tabular_data as
(
SELECT 
    job.job_id, 
    job.skill_id, 
    skill.skill_name
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
where job.job_skill_level_id=1
)

SELECT 
    job_id, 
    array_agg(distinct skill_id order by skill_id) as must_have_skill_ids,
    array_agg(distinct skill_name order by skill_name) as must_have_skill_names 
from tabular_data 
group by 1
"""


job_mcq_nice = """with tabular_data as
(
SELECT 
    job.job_id, 
    job.skill_id, 
    skill.skill_name
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
where job.job_skill_level_id=2
)

SELECT 
    job_id, 
    array_agg(distinct skill_id order by skill_id) as nice_have_skill_ids,
    array_agg(distinct skill_name order by skill_name) as nice_have_skill_names 
from tabular_data 
group by 1
"""

job_mcq_must = GBQ_data(job_mcq_must)
job_mcq_nice = GBQ_data(job_mcq_nice)
print(job_mcq_must.shape)
print(job_mcq_nice.shape)

job_mcq_must['must_have_skill_ids_tuple'] = job_mcq_must.must_have_skill_ids.apply(lambda x:tuple(x))
job_mcq_nice['nice_have_skill_ids_tuple'] = job_mcq_nice.nice_have_skill_ids.apply(lambda x:tuple(x))
job_mcq = job_mcq_must.merge(job_mcq_nice, how='outer', on='job_id')
must_have_mcq = job_mcq_must.groupby('must_have_skill_ids_tuple').agg({'job_id':'count'}).reset_index().rename(columns={'job_id':'num jobs'})
nice_have_mcq = job_mcq_nice.groupby('nice_have_skill_ids_tuple').agg({'job_id':'count'}).reset_index().rename(columns={'job_id':'num jobs'})

(8107, 3)
(4157, 3)


## Loading Data from r-pin
- devs-skill-pass-list

In [4]:
API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
dev_data = board.pin_read("muhammad_usman/devs-skill-pass-list")
dev_data['last_update_availability'] = pd.to_datetime(dev_data['last_update_availability'])
dev_data = dev_data.loc[(dev_data['num_passed_skill']>0) & (~dev_data['num_skill'].isna())]
dev_data = dev_data.loc[(dev_data['vetting_status'].isin(['available supply','ready 2.0','passed ti','standardized resume']))].reset_index(drop=True).copy()
dev_data['self_skill_id_list'] = dev_data.self_skill_id.apply(lambda x:list(map(int, x.split(',',-1))))
dev_data['passed_skill_id_list'] = dev_data.passed_skill_id.apply(lambda x:list(map(int, x.replace('[', '').replace(']','').split(',',-1))))
dev_data['passed_skill_id_set'] = dev_data.passed_skill_id_list.apply(lambda x:set(x))
dev_data['self_skill_id_set'] = dev_data.self_skill_id_list.apply(lambda x:set(x))
dev_data.shape

(37466, 48)

In [5]:

## for given skill or combination of skill what supply we have 

# input should be what we skill/list of skills 
# sorting is not working for now some reasons, 

start = time.time()

col_name = ['total_devs', 'Elite', 'High Quality', 'Average', 'Low Quality', 'Low Experience']
df_passed = pd.DataFrame(columns = col_name)
for i in must_have_mcq.index:
    df_passed = df_passed.append(avs_supply_passed_mcq(must_have_mcq.at[i, 'must_have_skill_ids_tuple']),ignore_index=True)
    
df_self = pd.DataFrame(columns = col_name)
for i in must_have_mcq.index:
    df_self = df_self.append(avs_supply_self_skill(must_have_mcq.at[i, 'must_have_skill_ids_tuple']),ignore_index=True)
    
df_passed.columns = ['passed_must_have_skill '+ val for val in list(df_passed.columns)]
df_self.columns = ['self_must_have_skill '+ val for val in list(df_self.columns)]
df = pd.concat([df_passed,df_self], axis=1)
df_must_have_mcq = pd.concat([must_have_mcq,df], axis=1)
df_final_must_have = job_mcq_must[['job_id','must_have_skill_ids_tuple']].merge(df_must_have_mcq, how='left', on='must_have_skill_ids_tuple')
job_mcq = job_mcq.merge(df_final_must_have[['job_id'] + list(df_passed.columns.append(df_self.columns))], how='left', on='job_id')
# Grab Currrent Time After Running the Code
end = time.time()
#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))


383.0870726108551


In [6]:
start = time.time()
# Code to time run
col_names = ['total_devs', 'Elite', 'High Quality', 'Average', 'Low Quality', 'Low Experience']
df_passed = pd.DataFrame(columns = col_names)
for i in nice_have_mcq.nice_have_skill_ids_tuple:
    df_passed = df_passed.append(avs_supply_passed_mcq(i),ignore_index=True)
    
df_self = pd.DataFrame(columns = col_names)
for i in nice_have_mcq.nice_have_skill_ids_tuple:
    df_self = df_self.append(avs_supply_self_skill(i),ignore_index=True)
    
df_passed.columns = ['passed_nice_have_skill '+ val for val in list(df_passed.columns)]
df_self.columns = ['self_nice_have_skill '+ val for val in list(df_self.columns)]
df = pd.concat([df_passed,df_self], axis=1)
df_nice_have_mcq = pd.concat([nice_have_mcq,df], axis=1)

df_final_nice_have = job_mcq_nice[['job_id','nice_have_skill_ids_tuple']].merge(df_nice_have_mcq, how='left', on='nice_have_skill_ids_tuple')

# Grab Currrent Time After Running the Code
end = time.time()
#Subtract Start Time from The End Time
total_time = end - start
print("\n"+ str(total_time))



201.6934576034546


In [7]:
job_mcq = job_mcq.merge(df_final_nice_have[['job_id'] + list(df_passed.columns.append(df_self.columns))], how='left', on='job_id')

In [10]:
API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
board.pin_write(job_mcq, "muhammad_usman/demand-avs-data", type="csv")
print(f'Pin muhammad_usman/demand-avs-data has been updated with shape {job_mcq.shape}')

Writing pin:
Name: 'muhammad_usman/demand-avs-data'
Version: 20220827T014644Z-be0e9


Pin muhammad_usman/demand-avs-data has been updated with shape (8115, 31)


In [ ]:
import os


In [6]:
!git clone git@github.com:TuringEnterprises/PDSA.git

Cloning into 'PDSA'...
key_load_public: invalid format
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 123 (delta 66), reused 97 (delta 49), pack-reused 0
Receiving objects: 100% (123/123), 6.01 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [3]:
%cd ..


/home/muhammad.usman/testuser3


In [4]:
!pwd

/home/muhammad.usman/testuser3
